# Binary Mix - Mesogens with NP's

In [1]:
#-----Import necessary packages for run the simulation.

from __future__ import division
import hoomd
import hoomd.md

#-----Define some important variables

t_max = 5.0;
p_0   = 0.9; 
p_1   = 1.1;
ramp_steps = 1e5;

#-----Create a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a gsd file.

system = hoomd.init.read_gsd("T_CM&NP_5.0_P_0.9_ramp.gsd", frame = -1);

#-----Define each mesogen in the local frame of reference in the rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Define the interaction between each constituent in the mix.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT.

press = hoomd.variant.linear_interp(points = [(0,p_0), (ramp_steps, p_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = t_max, tau = 0.5, tauP = 1.0, P = press);
npt.randomize_velocities(seed = 42)

#-----Save data in a .log and .gsd file.

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1004 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#-----Run the simulation.

hoomd.run(ramp_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 297182 / 397000 | TPS 18.171 | ETA 01:31:33
Time 00:00:20 | Step 297401 / 397000 | TPS 21.7213 | ETA 01:16:25
Time 00:00:30 | Step 297627 / 397000 | TPS 22.5653 | ETA 01:13:23
Time 00:00:40 | Step 297852 / 397000 | TPS 22.4656 | ETA 01:13:33
Time 00:00:50 | Step 298074 / 397000 | TPS 22.1546 | ETA 01:14:25
Time 00:01:00 | Step 298291 / 397000 | TPS 21.6944 | ETA 01:15:49
Time 00:01:10 | Step 298507 / 397000 | TPS 21.4364 | ETA 01:16:34
Time 00:01:20 | Step 298728 / 397000 | TPS 22.0073 | ETA 01:14:25
Time 00:01:30 | Step 298952 / 397000 | TPS 22.3844 | ETA 01:13:00
Time 00:01:40 | Step 299175 / 397000 | TPS 22.2656 | ETA 01:13:13
Time 00:01:50 | Step 299392 / 397000 | TPS 21.6976 | ETA 01:14:58
Time 00:02:00 | Step 299608 / 39

**ERROR**: nlist.tree(): Particle 337 is out of bounds (x: 2.81907, y: -1.00944, z: 16.2563, fx: 0.602312, fy: 0.463364, fz:1.08999)


RuntimeError: Error updating neighborlist

In [ ]:
#-----Display some information (density).

system.get_metadata()